In [1]:
import pandas as pd
import gc

## Загрузка данных

In [2]:
data = pd.read_csv("../ex04/fines.csv")

data

,CarNumber,Refund,Fines,Make,Model,Year
0,Y163O8161RUS,2.0,3200.0,Ford,Focus,1989
1,E432XX77RUS,1.0,6500.0,Toyota,Camry,1995
2,7184TT36RUS,1.0,2100.0,Ford,Focus,1984
3,X582HE161RUS,2.0,2000.0,Ford,Focus,2015
4,92918M178RUS,1.0,5700.0,Ford,Focus,2014
...,...,...,...,...,...,...
925,X123YZ77RUS,1.0,5000.0,Honda,Civic,1995
926,Y456AB77RUS,2.0,3200.0,Nissan,Sentra,2005
927,Z789CD77RUS,1.0,4100.0,Kia,Rio,2010
928,A012EF77RUS,2.0,2800.0,Hyundai,Elantra,2015


## Итерации

#### нужно вычислить fines / refund * year для каждой строки

- создать новый столбец с результатом

- измерить время выполнения каждой реализации с помощью магической команды `%%timeit`

#### Loop + iloc + append

- `iloc[]` - Извлекает строку по индексу

In [3]:
%%timeit
new_column = []

for i in range(len(data)):
    new_column.append(data.iloc[i]['Fines'] / data.iloc[i]['Refund'] * data.iloc[i]['Year'])

data['Calculated'] = new_column

111 ms ± 4.8 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


#### iterrows()

In [5]:
%%timeit

new_column = []

for _, row in data.iterrows():
    new_column.append(row['Fines'] / row['Refund'] * row['Year'])

data['Calculated'] = new_column

36.5 ms ± 789 μs per loop (mean ± std. dev. of 7 runs, 10 loops each)


#### apply() / lambda

In [7]:
%%timeit

data['Calculated'] = data.apply(lambda row: row['Fines'] / row['Refund'] * row['Year'], axis=1)

7.57 ms ± 179 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


#### Series

In [9]:
%%timeit

data['Calculated'] = (data['Fines'] / data['Refund'] * data['Year'])

183 μs ± 7.52 μs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


#### .values

In [11]:
%%timeit

fines = data['Fines'].values
refund = data['Refund'].values
year = data['Year'].values

data['Calculated'] = (fines / refund * year)

90.8 μs ± 1.75 μs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Индексация

- Получите строку по конкретному `CarNumber`, например `'O136HO197RUS'`.

- Установите `CarNumber` как индекс.

- Снова получите ту же строку по индексу.

- Get row

In [12]:
%%timeit

data.reset_index(inplace=True)

data.set_index('CarNumber', inplace=True)

car_row = data.loc['O136HO197RUS']

641 μs ± 14 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


- Re-set индекса

In [13]:
data.reset_index(inplace=True)
data.set_index('CarNumber', inplace=True)
car_row = data.loc['O136HO197RUS']

car_row

,index,Refund,Fines,Make,Model,Year,Calculated
CarNumber,,,,,,,
O136HO197RUS,715,2.0,7800.0,Toyota,Corolla,1999,7796100.0
O136HO197RUS,902,1.0,300.0,Toyota,Corolla,1991,597300.0


## Downcasting

- Запустите `df.info(memory_usage='deep')`. Обратите внимание на типы данных и объём памяти.

- Создайте копию *DataFrame*: `optimized = df.copy()`.

- Преобразуйте все столбцы `float64` в `float32`.

- Преобразуйте все `int64` в наименьший возможный числовой тип.

- Запустите снова `info(memory_usage='deep')` — сравните с изначальным *DataFrame*.

---

- `info` - который выводит информацию о структуре *DataFrame*

- `memory_usage='deep'` - заставляет pandas рассчитать реальный объем памяти - включая содержимое строковых данных

In [14]:
data.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
Index: 930 entries, Y163O8161RUS to B345GH77RUS
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   index       930 non-null    int64  
 1   Refund      930 non-null    float64
 2   Fines       930 non-null    float64
 3   Make        930 non-null    object 
 4   Model       919 non-null    object 
 5   Year        930 non-null    int64  
 6   Calculated  930 non-null    float64
dtypes: float64(3), int64(2), object(2)
memory usage: 221.5 KB



- `float64` в `float32`

---

- `to_numeric()` - Преобразует входные данные в числовой тип

- `downcast='float'` - Указывает, что нужно использовать наименьший возможный тип данных с плавающей точкой

In [15]:
data_optimized = data.copy()

data_optimized['Fines'] = pd.to_numeric(data_optimized['Fines'], downcast='float')
data_optimized['Refund'] = pd.to_numeric(data_optimized['Refund'], downcast='float')

data_optimized.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
Index: 930 entries, Y163O8161RUS to B345GH77RUS
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   index       930 non-null    int64  
 1   Refund      930 non-null    float32
 2   Fines       930 non-null    float32
 3   Make        930 non-null    object 
 4   Model       919 non-null    object 
 5   Year        930 non-null    int64  
 6   Calculated  930 non-null    float64
dtypes: float32(2), float64(1), int64(2), object(2)
memory usage: 214.3 KB


- `int64` в *наименьший возможный числовой тип*

In [16]:
data_optimized['Year'] = pd.to_numeric(data_optimized['Year'], downcast='integer')

data_optimized.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
Index: 930 entries, Y163O8161RUS to B345GH77RUS
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   index       930 non-null    int64  
 1   Refund      930 non-null    float32
 2   Fines       930 non-null    float32
 3   Make        930 non-null    object 
 4   Model       919 non-null    object 
 5   Year        930 non-null    int16  
 6   Calculated  930 non-null    float64
dtypes: float32(2), float64(1), int16(1), int64(1), object(2)
memory usage: 208.8 KB


## Категории

- Преобразуйте все столбцы с типом `object` в тип `category`.

- Проверьте объём памяти — он должен уменьшиться в 2–3 раза по сравнению с начальным вариантом.

In [17]:
data_optimized.reset_index(inplace=True)
data_optimized['CarNumber'] = data_optimized['CarNumber'].astype('category')

data_optimized.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 930 entries, 0 to 929
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   CarNumber   930 non-null    category
 1   index       930 non-null    int64   
 2   Refund      930 non-null    float32 
 3   Fines       930 non-null    float32 
 4   Make        930 non-null    object  
 5   Model       919 non-null    object  
 6   Year        930 non-null    int16   
 7   Calculated  930 non-null    float64 
dtypes: category(1), float32(2), float64(1), int16(1), int64(1), object(2)
memory usage: 171.2 KB


## Очистка памяти

- `gc.collect()` - Запускает сборщик мусора в *Python*, чтобы принудительно освободить память, занятую объектами, на которые больше нет ссылок

In [18]:
del data

gc.collect()

0